In [ ]:
# Libraries & Data - RUN FIRST - Code below is dependent on these libraries and variables
!pip install keras
!pip install tensorflow
!pip install numpy
!pip install pandas

from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers as Optimizer
import pandas as pd
from datetime import datetime

# Initialise Dataset
X = [0.0354, 1.0331], [-0.1302, 1.3282], [0.1665, 0.6873], [1.3223, -.1472], [0.0643, -.0250], [-0.1051,.9224]
Y = [.0, 1.0,0.0,.0],[.0,1.0,.0,.0], [.0,1.0,.0,.0], [.0,.0,1.0,.0], [1.0,.0,.0,.0], [.0,.0,.0,1.0]

# Initialise Test Data
testResult = []
testDP = 4

In [ ]:
# Algorithm which tests each combination of hyper parameter. Used in my ANN evaluatuion

# Hyper Parameters
ACT = ['relu', 'sigmoid', 'tanh', 'elu' ]
ACT_COUNT = len(ACT)

LR = [ 0.1, 0.2, 0.3, 0.5, 0.75, 1.0, 2.5, 5.0, 10.0 ]
LR_COUNT = len(LR)

LOSS = [ 'binary_crossentropy', 'mean_squared_error' , 'huber_loss', 'kullback_leibler_divergence'] 
LOSS_COUNT = len(LOSS)

OPT = [ Optimizer.Adadelta, Optimizer.Adam, Optimizer.Adagrad, Optimizer.Nadam, Optimizer.RMSprop]
OPT_COUNT = len(OPT)

EPOCH = [100] # [25, 50, 100]
EPOCH_COUNT = len(EPOCH)

TEST_MAX = (ACT_COUNT ** 2) * LR_COUNT * OPT_COUNT * LOSS_COUNT * EPOCH_COUNT

# Run tests for hyper parameter combinations
for epoch_i in range(0, EPOCH_COUNT, 1):
    for act_i in range(0, ACT_COUNT, 1):
        for act_j in range(0, ACT_COUNT, 1):
            for lr_i in range(0, LR_COUNT, 1):
                for opt_i in range(0, OPT_COUNT,1):
                    for loss_i in range(0, LOSS_COUNT,1):

                        # Current Model
                        model = Sequential()

                        # Add the first layer (input layer)
                        model.add(Dense(2, activation=ACT[act_i]))

                        # Add a second layer (hidden layer)
                        model.add(Dense(4, activation=ACT[act_i]))

                        # Add the output layer
                        model.add(Dense(4, activation=ACT[act_j]))

                        # Compile the model
                        model.compile(loss = LOSS[loss_i], optimizer = OPT[opt_i](learning_rate= LR[lr_i]), metrics = ['accuracy'])

                        # Train the model with epochs, batch size, and validation data
                        model.fit(X, Y, epochs=EPOCH[epoch_i], batch_size=32, verbose = 0)

                        # Evaluate the model testing set and get the loss and accuracy
                        loss, accuracy = model.evaluate(X,Y)

                        # Save test results
                        testResult.append({ 
                            'loss': round(loss, testDP),
                            'accuracy': round(accuracy, testDP),
                            'epochs': EPOCH[epoch_i],
                            'act_hid': ACT[act_i],
                            'act_out': ACT[act_j],
                            'learn rate': LR[lr_i],
                            'optimizer': model.optimizer.get_config()['name'],
                        })
                            
                        print('Test {}/{} complete'.format(len(testResult), TEST_MAX))

# Export test results to csv
dateTimeString = datetime.now().strftime("%d-%m-%Y %H-%M-%S")
pd.DataFrame(testResult).to_csv('Test Hyper {}.csv'.format(dateTimeString), sep=',', index=False)

In [ ]:
# Algorithm which tests for the best number of neurons and layers to use 

# Test Parameters
layerMax = 5
neuronMax = 21
testMax = layerMax * neuronMax

for l in range(1, layerMax, 1):
    for n in range(2, neuronMax, 1):

        # Create a Sequential model
        model = Sequential()

        # Add our hidden layers
        for i in range(0, l, 1):
            model.add(Dense(n, activation='elu'))

        # Add the output layer
        model.add(Dense(4, activation='sigmoid'))

        # Compile the model
        # We'll use the binary crossentropy loss function, the Adam optimizer, and accuracy as the metric
        model.compile(loss='binary_crossentropy', optimizer=Optimizer.Nadam(0.1), metrics=['accuracy'])

        # Assuming X_train and y_train are your training data and labels
        model.fit(X, Y, epochs=100, batch_size=32)

        # Evaluate the model testing set and get the loss and accuracy
        loss, accuracy = model.evaluate(X,Y)

        testResult.append({'loss': round(loss, testDP), 'accuracy': round(accuracy, testDP), 'Layers': l, 'neurons': n})

        print('Test {}/{} complete'.format(len(testResult), testMax))

# Export test results to csv
dateTimeString = datetime.now().strftime("%d-%m-%Y %H-%M-%S")
pd.DataFrame(testResult).to_csv('Test Neurons {}.csv'.format(dateTimeString), sep=',', index=False)

In [ ]:
# My Optimal Model
neruonCount = 6
layerCount = 2
# Create a Sequential model
model = Sequential()

# Add a second layer (hidden layer)
for i in range(0, layerCount, 1):
    model.add(Dense(6, activation='elu'))

# Add the output layer
model.add(Dense(4, activation='sigmoid'))

# Compile the model
# We'll use the binary crossentropy loss function, the Adam optimizer, and accuracy as the metric
model.compile(loss='binary_crossentropy', optimizer=Optimizer.Nadam(0.1), metrics=['accuracy'])

# Assuming X_train and y_train are your training data and labels
model.fit(X, Y, epochs=50, batch_size=32)

loss, accuracy = model.evaluate(X,Y)

print('Loss:', round(loss, testDP), 'Accuracy:', round(accuracy, testDP))